In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('jobOffers.csv', index_col=0, skipinitialspace=True)


In [3]:
#salary
df = df.drop(df[df['salaryRange'] == 'neplacená stáž'].index)
df['salLower'] = df['salaryRange'].str.split(" - ").str[0].str.split("  ").str[0]
df['salUpper'] = df['salaryRange'].str.split(" - ").str[1].str.split("  ").str[0]

df['salLower'] = df['salLower'].replace({'k': '*1e3'}, regex=True).map(pd.eval).astype(int)
df['salUpper'].fillna(df['salLower'], inplace = True)
df['salUpper'] = df['salUpper'].replace({'k': '*1e3'}, regex=True).map(pd.eval).astype(int)

df['salAvg'] = (df['salUpper'] + df['salLower'])/2



In [4]:
#category and main programming language
df['category+progLang'] = df['category+progLang'].str.replace("Kategorie:","").str.strip()
df['category'] = df['category+progLang'].str.split(",").str[0]
df['mainProgLang'] = df['category+progLang'].str.split(",").str[1]


In [5]:
#seniority
df["seniority"] = df["seniority"].str.strip().replace({'Trainee, Junior': 'Junior', 'Trainee': 'Junior',
                                                          'Senior, Expert':'Senior',
                                                          'Expert':'Senior', 'Mid, Senior':'Mid Advanced'})


In [6]:
#Work language
df["workLanguage"]=df["workLanguage"].str.replace("Work language:","")


In [7]:
#getting rid of the numbers, dots and converting to lowercase to standardise prog langugage
df.loc[:,"mainProgLang"] = df.loc[:,"mainProgLang"].str.replace('\d+', '')
df.loc[:,"mainProgLang"] = df.loc[:,"mainProgLang"].str.replace('.', '')
df["mainProgLang"]=df["mainProgLang"].str.lower()
#converting less demanded programming languages to 'other'
mask = df['mainProgLang'].map(df['mainProgLang'].value_counts())<16
df['mainProgLangAdj']=df['mainProgLang'].mask(mask,'other')
counts = df.groupby(['mainProgLangAdj']).size().sort_values(ascending=False)
print(counts)

mainProgLangAdj
other          211
 java          121
 python         85
 javascript     73
 net            63
 react          62
 php            46
 node           21
dtype: int64


<ipython-input-7-5ea043900ff0>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:,"mainProgLang"] = df.loc[:,"mainProgLang"].str.replace('\d+', '')
<ipython-input-7-5ea043900ff0>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.loc[:,"mainProgLang"] = df.loc[:,"mainProgLang"].str.replace('.', '')


In [8]:
#leaving just first adress in location for simplification
#df["location"]=df["location"].str.split(",").str[0]
#df["location"]=df["location"].str.split("•").str[0]
#adjusting remote with location -- Where is Na dálku in remote => location: Remote, 
#when remote: Nan or Covid-time na dálku => location: first city witout street adress
#df.loc[df.remote == "Na dálku ","location"]= "Remote"
df.loc[df.remote != "Na dálku ","remote"]= "Hybrid or no remote"
df['remoteAdj'] = ['Remote' if x == 'Na dálku ' else 'Hybrid or no remote' for x in df['remote']]



In [9]:
#Work language
df["workLanguage"]=df["workLanguage"].str.replace("Work language:","")
df['language'] = df['workLanguage'].str.split(',')

df.groupby(['workLanguage']).size()
langList = ['angličtina','česky']
for lang in langList:
    df[lang] = df['workLanguage'].str.contains(lang)

#['angličtina', 'česky' ]

In [10]:
#Because we are interested only on the job offers relevant for the Czech people, we are removing all jobs where neither Czech nor English is used as a speaking language.
df = df[(df['angličtina'] == True)| (df['česky'] == True)]

In [11]:
#do we keep bonus? (and jobTitles and company?)
df.groupby("Bonus").size()

#base on this we decide to not include bonus in the final dataset

Bonus
+10%       25
+12%        1
+13%        1
+15%        3
+20%        2
+30%        3
+5%         4
+50%        2
+7%         1
20k CZK     1
dtype: int64

In [12]:
#controlling for missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 659 entries, 0 to 719
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   jobTitles          659 non-null    object 
 1   company            659 non-null    object 
 2   salaryRange        659 non-null    object 
 3   Bonus              43 non-null     object 
 4   category+progLang  659 non-null    object 
 5   seniority          659 non-null    object 
 6   workLanguage       659 non-null    object 
 7   location           549 non-null    object 
 8   remote             659 non-null    object 
 9   salLower           659 non-null    int64  
 10  salUpper           659 non-null    int64  
 11  salAvg             659 non-null    float64
 12  category           659 non-null    object 
 13  mainProgLang       624 non-null    object 
 14  mainProgLangAdj    624 non-null    object 
 15  remoteAdj          659 non-null    object 
 16  language           659 non

In [13]:
#subseting dataset
df_final=df[['jobTitles','seniority', 'workLanguage','angličtina', 'česky','location','remoteAdj','salAvg', 'category',
       'mainProgLangAdj']]
df_final.head()


,jobTitles,seniority,workLanguage,angličtina,česky,location,remoteAdj,salAvg,category,mainProgLangAdj
0,Java Developer,Mid Advanced,česky,False,True,"Hybrid +1 • Praha 40% on-site, 60% remotely",Hybrid or no remote,136500.0,Backend,java
3,Python / Delphi Developer,Mid,česky,False,True,"Hybrid +1 • Žďár nad Sázavou , Smetanova 116...",Hybrid or no remote,47500.0,Backend,python
4,3d Graphics Developer,Mid,česky,False,True,Prague (Po pandemi) • Prague,Hybrid or no remote,89250.0,Backend,other
5,(Senior) Software Developer,Senior,"angličtina, německy",True,False,Hybrid +1 • Niederdreisbach,Hybrid or no remote,135850.0,Backend,python
6,Senior Backend Developer,Senior,"česky, angličtina",True,True,"Hybrid +1 • Praha , Klimentská 36",Hybrid or no remote,65000.0,Backend,NaN


In [14]:
#making sure there are no whitespaces left
df_final.columns = df_final.columns.str.strip()



In [15]:
#to CSV
df_final.to_csv("cleaned_data.csv")